In [1]:
import numpy as np
import os
import struct


def load_mnist(path='dataset', kind='train'):
    """
    A helper function to load the MNIST dataset.
    :param path:
    :param kind:
    :return: images, labels
    """
    path = os.path.join(os.getcwd(), path)
    labels_path = os.path.join(path, '%s-labels.idx1-ubyte' % kind)
    images_path = os.path.join(path, '%s-images.idx3-ubyte' % kind)
    with open(labels_path, 'rb') as lbpath:
        magic, n_labels = struct.unpack('>II', lbpath.read(8))
        labels = np.fromfile(lbpath, dtype=np.uint8)
    with open(images_path, 'rb') as imgpath:
        magic, n_images, rows, columns = struct.unpack('>IIII', imgpath.read(16))
        images = np.fromfile(imgpath, dtype=np.uint8).reshape(n_images, rows * columns)
    print('Load %s data, images rows: %d, label rows: %d' % (kind, n_images, n_labels))
    return images, labels


x_train, y_train = load_mnist(kind='train')
x_test, y_test = load_mnist(kind='t10k')

Load train data, images rows: 60000, label rows: 60000
Load t10k data, images rows: 10000, label rows: 10000


In [8]:
mean_val = np.mean(x_train, axis=0)
std_val = np.std(x_train)
x_train_centered = (x_train - mean_val) / std_val
x_test_centered = (x_test - mean_val) / std_val
del x_train, x_test

In [9]:
def batch_generator(X, y, batch_size=100, shuffle=False, random_seed=None):
    """
    A helper function to generate batches of instances.
    :param X: 
    :param y: 
    :param batch_size: 
    :param shuffle: 
    :param random_seed: 
    :return: 
    """
    if shuffle:
        idx = np.arange(y.shape[0])
        random = np.random.RandomState(random_seed)
        random.shuffle(idx)
        X = X[idx]
        y = y[idx]
    for i in range(0, X.shape[0], batch_size):
        yield (X[i:i + batch_size, :], y[i:i + batch_size])

In [13]:
import tensorflow as tf


class ConvNNModel(object):
    def __init__(self, batch_size=100, epochs=20, dropout_rate=0.5,
                 learning_rate=1e-4, shuffle=True, random_seed=None):
        self.batch_size = batch_size
        self.epochs = epochs
        self.dropout_rate = dropout_rate
        self.learning_rate = learning_rate
        self.shuffle = shuffle
        self.random_seed = random_seed
        np.random.seed(random_seed)
        g = tf.Graph()
        with g.as_default():
            tf.set_random_seed(random_seed)
            self.build()
            self.init_op = tf.global_variables_initializer()
            self.saver = tf.train.Saver()
        self.sess = tf.Session(graph=g)
        self.sess.run(self.init_op)

    def build(self):
        tf_x = tf.placeholder(tf.float32, shape=[None, 784], name='tf_x')
        tf_y = tf.placeholder(tf.int32, shape=[None], name='tf_y')
        is_train = tf.placeholder(tf.bool, shape=(), name='is_train')
        # reshape tf_x to 2d image
        tf_x_2dimage = tf.reshape(tf_x, shape=[-1, 28, 28, 1])
        # one hot coding on tf_y
        y_onehot = tf.one_hot(indices=tf_y, depth=10, dtype=tf.float32)

        # 1st layer: conv2d layer
        h1 = tf.layers.conv2d(tf_x_2dimage, kernel_size=[5, 5], filters=32,
                              padding='valid', activation=tf.nn.relu, name='h1')
        # max pooling
        h1_pool = tf.layers.max_pooling2d(h1, pool_size=[2, 2], strides=2, name='h1_pool')
        print(h1)
        print(h1_pool)

        # 2nd layer: con2d layer
        h2 = tf.layers.conv2d(h1_pool, kernel_size=[5, 5], filters=64,
                              padding='valid', activation=tf.nn.relu, name='h2')
        # max pooling
        h2_pool = tf.layers.max_pooling2d(h2, pool_size=[2, 2], strides=2, name='h2_pool')
        print(h2)
        print(h2_pool)

        # 3rd layer: dense
        input_shape = h2_pool.get_shape().as_list()
        n_input_units = np.prod(input_shape[1:])
        h2_pool_flat = tf.reshape(h2_pool, shape=[-1, n_input_units], name='h2_pool_flat')
        print(h2_pool_flat)
        h3 = tf.layers.dense(h2_pool_flat, units=1024, activation=tf.nn.relu, name='h3')
        # dropout
        h3_drop = tf.layers.dropout(h3, training=is_train, rate=self.dropout_rate, name='h3_drop')
        print(h3)
        print(h3_drop)

        # 4th layer: dense
        logits = tf.layers.dense(h3_drop, units=10, activation=None, name='logits')
        print(logits)

        # prediction
        prediction_labels = tf.cast(tf.argmax(logits, axis=1), tf.int32, name='prediction_labels')
        print(prediction_labels)
        # accuracy
        correct = tf.equal(prediction_labels, tf_y)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name='accuracy')
        # loss function
        loss = tf.losses.softmax_cross_entropy(onehot_labels=y_onehot, logits=logits)
        print(loss)
        # optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate)
        train_op = optimizer.minimize(loss=loss, name='train_op')

    def train(self, training_set, validation_set):
        x_data = np.array(training_set[0])
        y_data = np.array(training_set[1])
        for epoch in range(1, self.epochs + 1):
            batch = batch_generator(x_data, y_data, batch_size=self.batch_size, 
                                    shuffle=self.shuffle, random_seed=self.random_seed)
            for _, (x, y) in enumerate(batch):
                feed_dict = {'tf_x:0': x, 'tf_y:0': y, 'is_train:0': True}
                self.sess.run('train_op', feed_dict=feed_dict)
            x_validation = np.array(validation_set[0])
            y_validation = np.array(validation_set[1])
            feed_dict = {'tf_x:0': x_validation, 'tf_y:0': y_validation, 'is_train:0': False}
            validation_accuracy = self.sess.run('accuracy:0', feed_dict=feed_dict)
            print("Epoch: %2d validation accuracy: %6.2f" % (epoch, (100 * validation_accuracy)))

    def predict(self, x):
        x = np.array(x)
        feed_dict = {'tf_x:0': x, 'is_train:0': False}
        predict_labels = self.sess.run('prediction_labels:0', feed_dict=feed_dict)
        return predict_labels

    def save(self, epoch, path='./model/cnn/'):
        if not os.path.isdir(path):
            os.makedirs(path)
        self.saver.save(self.sess, os.path.join(path, 'model.ckpt'), global_step=epoch)
        print('Model saved in %s' % path)

In [14]:
model = ConvNNModel(random_seed=123)

Tensor("h1/Relu:0", shape=(?, 24, 24, 32), dtype=float32)
Tensor("h1_pool/MaxPool:0", shape=(?, 12, 12, 32), dtype=float32)
Tensor("h2/Relu:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("h2_pool/MaxPool:0", shape=(?, 4, 4, 64), dtype=float32)
Tensor("h2_pool_flat:0", shape=(?, 1024), dtype=float32)
Tensor("h3/Relu:0", shape=(?, 1024), dtype=float32)
Tensor("h3_drop/cond/Merge:0", shape=(?, 1024), dtype=float32)
Tensor("logits/BiasAdd:0", shape=(?, 10), dtype=float32)
Tensor("prediction_labels:0", shape=(?,), dtype=int32)
Tensor("softmax_cross_entropy_loss/value:0", shape=(), dtype=float32)


In [15]:
model.train(training_set=(x_train_centered, y_train), validation_set=(x_test_centered, y_test))

Epoch:  1 validation accuracy:  97.38
Epoch:  2 validation accuracy:  98.25
Epoch:  3 validation accuracy:  98.58
Epoch:  4 validation accuracy:  98.76
Epoch:  5 validation accuracy:  98.95
Epoch:  6 validation accuracy:  98.96
Epoch:  7 validation accuracy:  99.00
Epoch:  8 validation accuracy:  99.06
Epoch:  9 validation accuracy:  99.19
Epoch: 10 validation accuracy:  99.13
Epoch: 11 validation accuracy:  99.12
Epoch: 12 validation accuracy:  99.24
Epoch: 13 validation accuracy:  99.09
Epoch: 14 validation accuracy:  99.26
Epoch: 15 validation accuracy:  99.30
Epoch: 16 validation accuracy:  99.27
Epoch: 17 validation accuracy:  99.13
Epoch: 18 validation accuracy:  99.25
Epoch: 19 validation accuracy:  99.15
Epoch: 20 validation accuracy:  99.23


In [16]:
model.save(epoch=model.epochs)

Model saved in ./model/cnn/
